In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 403.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 9.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 11.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 12.0 MB/s eta 0:00:00


In [2]:
from transformers import pipeline
import mlflow
from mlflow import MlflowClient
import torch
import json
import os

2024-01-23 12:10:14.879485: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-23 12:10:16.464507: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
qa = pipeline(
    'question-answering',
    model='./distilbert_bertqa',
    device=-1 #0->GPU
)

In [4]:
context = "I think Phoenix will be so successful, don't you think?"
question = "What does she said about Phoenix?"
qa(context=context, question=question)

{'score': 0.6935740113258362,
 'start': 8,
 'end': 37,
 'answer': 'Phoenix will be so successful'}

In [15]:
class DistilBERTModel(mlflow.pyfunc.PythonModel):

    def _preprocess(self, inputs):
        context = inputs['context']
        question = inputs['question']
        return context, question
        
    def load_context(self, device=-1):
        self.model = pipeline(
            'question-answering',
             model='./distilbert_bertqa',
             device=device
        )
        print('Carreguei')
        
    def predict(self, inputs):
        context, question = _preprocess(inputs)
        output = self.model(context, question)
        return output

    @classmethod
    def log_model(cls, model, base_path, model_name): #eg (model, '', 'my_model')
        torch_model_path = os.path.join(base_path, model_name)
        requirements = [
            "transformers==4.37.0",
            "mlflow==2.5.0",
            "numpy==1.24.3",
            "torch==2.0.0",
            "tqdm==4.65.0",
        ]
        mlflow.pyfunc.log_model(
            model_name,
            python_model=cls(),
            artifacts={"model": model_name},
            pip_requirements=requirements
        )

In [16]:
mlflow.set_experiment(experiment_name='test_bertqa')

<Experiment: artifact_location='/phoenix/mlflow/504731225219629820', creation_time=1705969426272, experiment_id='504731225219629820', last_update_time=1705969426272, lifecycle_stage='active', name='test_bertqa', tags={}>

In [18]:
with mlflow.start_run(run_name='distilbert_bertqa') as run:
    print(run.info.run_id)
    DistilBERTModel.log_model(model=qa, base_path='./', model_name='distilbert_bertqa')
    mlflow.register_model(model_uri = f"runs:/{run.info.run_id}/distilbert_bertqa", name="distilbert_bertqa")

3e5c614726c643d992d06726a14fcfd9


Registered model 'distilbert_bertqa' already exists. Creating a new version of this model...
2024/01/23 12:22:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: distilbert_bertqa, version 3
Created version '3' of model 'distilbert_bertqa'.


In [19]:
run.info

<RunInfo: artifact_uri='/phoenix/mlflow/504731225219629820/3e5c614726c643d992d06726a14fcfd9/artifacts', end_time=None, experiment_id='504731225219629820', lifecycle_stage='active', run_id='3e5c614726c643d992d06726a14fcfd9', run_name='distilbert_bertqa', run_uuid='3e5c614726c643d992d06726a14fcfd9', start_time=1706012563077, status='RUNNING', user_id='root'>

In [ ]:
!mlflow models serve -m /phoenix/mlflow/504731225219629820/3e5c614726c643d992d06726a14fcfd9/artifacts/distilbert_bertqa --env-manager conda --port 5001

/opt/conda/lib/python3.10/site-packages/click/core.py:2362: UserWarning: Use of conda is discouraged. If you use it, please ensure that your use of conda complies with Anaconda's terms of service (https://legal.anaconda.com/policies/en/?name=terms-of-service). virtualenv is the recommended tool for environment reproducibility. To suppress this warning, set the MLFLOW_DISABLE_ENV_MANAGER_CONDA_WARNING environment variable to 'TRUE'.
  value = self.callback(ctx, self, value)
2024/01/23 12:23:10 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/01/23 12:23:12 INFO mlflow.utils.conda: === Creating conda environment mlflow-cb3037257d688284e46da00970c6ac7ed000bf81 ===
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Installing pip dependencies: ...working... 